In [1]:
# Cell 1: Update transformers
!pip install --upgrade transformers
!pip install git+https://github.com/huggingface/transformers.git
# First, let's install a specific version
!pip install -q -U "transformers>=4.42.0" accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8fd5sof1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8fd5sof1
  Resolved https://github.com/huggingface/transformers.git to commit ff13eb668aa03f151ded71636d723f2e490ad967
  Installing build dependencies ... done
  Getting requirements to bu

In [2]:
!pip install torch torchvision
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 51.6 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import json
from PIL import Image

# Define all file paths
BANGLA_QUESTIONS_PATH = "/kaggle/input/test-nlp/Bangla_Train_Ques_220K.json"
BANGLA_ANSWERS_PATH = "/kaggle/input/test-nlp/Bangla_Train_Ans_220K.json"
IMAGE_DIR = "/kaggle/input/nlp-image/train2014"

print("Loading data files...")

# Load Bangla questions and answers
with open(BANGLA_QUESTIONS_PATH, 'r', encoding='utf-8') as f:
    bangla_questions = json.load(f)

with open(BANGLA_ANSWERS_PATH, 'r', encoding='utf-8') as f:
    bangla_answers = json.load(f)

print(f"Loaded {len(bangla_questions)} questions and {len(bangla_answers)} answers")

# Create mapping from question_id to answer
question_id_to_answer = {}
for answer in bangla_answers:
    question_id_to_answer[answer['question_id']] = answer['multiple_choice_answer']

print(f"Created answer mapping for {len(question_id_to_answer)} questions")

# Get all available image files
print(f"\nScanning image directory: {IMAGE_DIR}")
image_files = os.listdir(IMAGE_DIR)
print(f"Found {len(image_files)} images in train2014 directory")

# Create a mapping from image_id to filename by extracting IDs from filenames
image_id_to_filename = {}
for filename in image_files:
    if filename.endswith('.jpg'):
        parts = filename.split('_')
        if len(parts) >= 3:
            image_id_str = parts[-1].replace('.jpg', '')
            try:
                image_id = int(image_id_str)
                image_id_to_filename[image_id] = filename
            except ValueError:
                continue

print(f"Created image ID mapping for {len(image_id_to_filename)} images")

# --- MODIFIED SECTION ---

# Create test data
print("\nCreating test data... (This may take a while)")
test_data = []
successful_samples = 0
failed_samples = 0
failed_alternative_samples = 0

# Use enumerate for progress tracking
# REMOVED the [:50] slice to process ALL questions
for i, question in enumerate(bangla_questions): 
    image_id = question['image_id']
    question_id = question['question_id']
    question_text = question['question']
    
    # Get the answer
    answer = question_id_to_answer.get(question_id, "Unknown")
    
    # Get the image filename
    image_filename = image_id_to_filename.get(image_id)
    
    image_path = None
    
    if image_filename:
        # Primary check
        image_path = os.path.join(IMAGE_DIR, image_filename)
        if not os.path.exists(image_path):
            # Only print the first 5 errors to avoid spam
            if failed_samples < 5:
                print(f"Image path doesn't exist: {image_path}")
            failed_samples += 1
            image_path = None # Reset path
            
    else:
        # Try alternative: search for any image with this ID pattern
        search_id = f"{image_id:012d}"
        found_alternative = None
        for filename in image_files:
            if search_id in filename:
                found_alternative = filename
                break
        
        if found_alternative:
            image_path = os.path.join(IMAGE_DIR, found_alternative)
        else:
            # Only print the first 5 errors to avoid spam
            if failed_alternative_samples < 5:
                 print(f"Failed to find any image for ID {image_id} (search key: {search_id})")
            failed_alternative_samples += 1

    # If we successfully found an image path (either way)
    if image_path:
        sample = {
            "id": f"bangla_vqa_{question_id}",
            "image": image_path, # Use the verified path
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question_text}"
                },
                {
                    "from": "gpt", 
                    "value": answer
                }
            ]
        }
        test_data.append(sample)
        successful_samples += 1
    
    # --- Progress Tracker ---
    if (i + 1) % 10000 == 0:
        print(f"  ... Processed {i + 1} / {len(bangla_questions)} questions")

# --- END OF MODIFIED SECTION ---

print(f"\nSuccessfully created {successful_samples} test samples")
print(f"Failed (path invalid): {failed_samples} samples")
print(f"Failed (no image found): {failed_alternative_samples} samples")

# Save the test data
if test_data:
    output_filename = 'bangla_vqa_train_full.json'
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=2)
    print(f"Full training data saved as '{output_filename}'")

    # Show sample of created data
    print("\nSample created test data:")
    for i, sample in enumerate(test_data[:3]):
        question_text = sample['conversations'][0]['value'].replace('<image>\n', '')
        print(f"\nSample {i+1}:")
        print(f"  Image: {os.path.basename(sample['image'])}")
        print(f"  Question: {question_text}")
        print(f"  Answer: {sample['conversations'][1]['value']}")
else:
    print("ERROR: No test data was created!")

Loading data files...
Loaded 220000 questions and 220000 answers
Created answer mapping for 220000 questions

Scanning image directory: /kaggle/input/nlp-image/train2014
Found 69933 images in train2014 directory
Created image ID mapping for 69933 images

Creating test data... (This may take a while)
Failed to find any image for ID 524291 (search key: 000000524291)
Failed to find any image for ID 524291 (search key: 000000524291)
Failed to find any image for ID 524291 (search key: 000000524291)
Failed to find any image for ID 393224 (search key: 000000393224)
Failed to find any image for ID 393224 (search key: 000000393224)
  ... Processed 10000 / 220000 questions
  ... Processed 20000 / 220000 questions
  ... Processed 30000 / 220000 questions
  ... Processed 40000 / 220000 questions
  ... Processed 50000 / 220000 questions
  ... Processed 60000 / 220000 questions
  ... Processed 70000 / 220000 questions
  ... Processed 80000 / 220000 questions
  ... Processed 90000 / 220000 questions


In [4]:
!pip install fuzzywuzzy[levenshtein]

In [5]:
import torch
import sys
import os
import json
import base64
import csv
from io import BytesIO
from PIL import Image
import transformers
from fuzzywuzzy import fuzz
from IPython.display import display, HTML, Image as IPImage

# --- [STEP 1/2] Load Model and Processor ---
print(f"--- Loading Model and Processor... ---")
print(f"Using transformers version: {transformers.__version__}")

try:
    from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
    print("✓ Successfully imported Qwen2VLForConditionalGeneration")
except ImportError as e:
    print(f"FATAL: Could not import Qwen2VL. Did you restart the runtime?")
    print(f"Error: {e}")
    sys.exit(1)

# Model configuration
model_id = "Qwen/Qwen2-VL-7B-Instruct"
test_file = "bangla_vqa_train_full.json" # Ensure path matches your output from Part 1
N_SAMPLES = 150
N_PRINT_SAMPLES = 10  # Only print details for the first 10
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

if not os.path.exists(test_file):
    print(f"FATAL ERROR: File not found: {test_file}")
    # In a notebook, we might not want to exit strictly, but here it's safer
    # sys.exit() 

# Load processor
print("Loading processor...")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print("✓ Processor loaded successfully")

# Load model
print("Loading model...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)
print("✓ Model loaded successfully.")

# --- [STEP 2/2] Load Test Data and Run Inference ---
print(f"\n--- Loading test data and running inference... ---")
with open(test_file, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

N_SAMPLES = min(N_SAMPLES, len(test_data))
test_samples = test_data[:N_SAMPLES]
print(f"Running inference on {len(test_samples)} Bangla VQA samples...")

correct_predictions = 0
total_similarity_score = 0
results = []

for i, sample in enumerate(test_samples):
    try:
        image_path = sample['image']
        question = sample['conversations'][0]['value'].replace('<image>\n', '').strip()
        ground_truth_answer = sample['conversations'][1]['value']
        
        if not os.path.exists(image_path):
            print(f"SKIPPING: Image not found at {image_path}")
            continue
            
        image = Image.open(image_path).convert("RGB")
        
        # Bangla Prompt
        bangla_prompt = f"""ছবির উপর ভিত্তি করে এই প্রশ্নের উত্তর দিন।

প্রশ্ন: {question}
উত্তর (শুধুমাত্র ১-২ শব্দে):"""

        messages = [
            {
                "role": "user", 
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": bangla_prompt}
                ]
            }
        ]
        
        text = processor.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        inputs = processor(
            text=text, 
            images=image, 
            return_tensors="pt"
        ).to(device)

        # Generate response
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=50, 
                do_sample=False
            )
        
        # Decode
        generated_ids = generated_ids[:, inputs['input_ids'].shape[1]:]
        model_answer = processor.decode(
            generated_ids[0], 
            skip_special_tokens=True
        ).strip().split('\n')[0] 

        # Fuzzy Matching
        gt_lower = ground_truth_answer.lower()
        pred_lower = model_answer.lower()
        similarity_score = fuzz.partial_ratio(gt_lower, pred_lower)
        total_similarity_score += similarity_score

        is_correct = similarity_score > 85
        if is_correct:
            correct_predictions += 1
            
        # Store results
        results.append({
            'image_path': image_path,
            'question': question,
            'true_answer': ground_truth_answer,
            'model_answer': model_answer,
            'similarity_score': similarity_score,
            'correct': is_correct
        })

        # Print rich output (limited)
        if i < N_PRINT_SAMPLES:
            print(f"\n--- Sample {i+1}/{N_SAMPLES} ---")
            # display(IPImage(filename=image_path, width=300)) # Uncomment in Jupyter
            print(f"Question: {question}")
            print(f"True Answer: {ground_truth_answer}")
            print(f"Model Answer: {model_answer}")
            print(f"Score: {similarity_score}%")

    except Exception as e:
        print(f"\n--- Sample {i+1}/{N_SAMPLES} ---")
        print(f"SKIPPING: Error processing sample: {e}")

# --- [STEP 3/3] Final Report & Export ---
print("\n--- [FINAL REPORT] ---")
accuracy = (correct_predictions / N_SAMPLES) * 100 if N_SAMPLES > 0 else 0
avg_similarity = total_similarity_score / N_SAMPLES if N_SAMPLES > 0 else 0

print(f"Total Samples: {N_SAMPLES}")
print(f"Correct (Score > 85%): {correct_predictions}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Average Answer Similarity: {avg_similarity:.2f}%")

# 1. Save JSON
results_file = "bangla_vqa_results_detailed.json"
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump({
        'accuracy': accuracy,
        'avg_similarity': avg_similarity,
        'correct_predictions': correct_predictions,
        'total_samples': N_SAMPLES,
        'detailed_results': results
    }, f, ensure_ascii=False, indent=4)
print(f"\nDetailed JSON results saved to {results_file}")

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


--- Loading Model and Processor... ---
Using transformers version: 5.0.0.dev0
✓ Successfully imported Qwen2VLForConditionalGeneration
Using device: cuda
Loading processor...


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/730 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

✓ Model loaded successfully.

--- Loading test data and running inference... ---


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Running inference on 150 Bangla VQA samples...

--- Sample 1/150 ---
Question: এই ছবিটি কিসের মাধ্যমে তোলা হয়েছে?
True Answer: নেট
Model Answer: স্টেডিয়াম থেকে
Score: 33%

--- Sample 2/150 ---
Question: এই লোকটি কোন অবস্থানে খেলছে?
True Answer: কলস
Model Answer: বেসবল টেম্পলেট
Score: 33%

--- Sample 3/150 ---
Question: খেলোয়াড়দের শার্টের রং কি?
True Answer: কমলা
Model Answer: রাখা হয়েছে না।
Score: 25%

--- Sample 4/150 ---
Question: এই লোকটি কি একজন পেশাদার বেসবল খেলোয়াড়?
True Answer: হ্যাঁ
Model Answer: হ্যাঁ
Score: 100%

--- Sample 5/150 ---
Question: তুষার কি রঙ?
True Answer: সাদা
Model Answer: সাদা
Score: 100%

--- Sample 6/150 ---
Question: ব্যক্তি কি করছে?
True Answer: স্কিইং
Model Answer: স্কি করছে।
Score: 67%

--- Sample 7/150 ---
Question: ব্যক্তিদের হেডওয়্যারের রঙ কী?
True Answer: লাল
Model Answer: হেডওয়্যার লাল।
Score: 100%

--- Sample 8/150 ---
Question: আকাশ কি নীল?
True Answer: হ্যাঁ
Model Answer: নিশ্চিত নয়
Score: 22%

--- Sample 9/150 ---
Question: পাহাড়ে কি 

In [6]:
!pip install XlsxWriter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.6 MB/s eta 0:00:0000:01


In [7]:
import xlsxwriter

# Define output filename
excel_filename = "bangla_vqa_results_with_images.xlsx"
print(f"Creating Excel file with embedded images: {excel_filename}...")

# Create the workbook and worksheet
workbook = xlsxwriter.Workbook(excel_filename)
worksheet = workbook.add_worksheet()

# Define formats
header_format = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3', 'border': 1})
cell_format = workbook.add_format({'text_wrap': True, 'valign': 'top', 'border': 1})
status_correct = workbook.add_format({'font_color': 'green', 'bold': True, 'border': 1, 'valign': 'top'})
status_wrong = workbook.add_format({'font_color': 'red', 'bold': True, 'border': 1, 'valign': 'top'})

# Set column widths
worksheet.set_column('A:A', 20)  # Image column
worksheet.set_column('B:B', 30)  # Question
worksheet.set_column('C:C', 20)  # True Answer
worksheet.set_column('D:D', 20)  # Model Answer
worksheet.set_column('E:E', 15)  # Category (New Column)
worksheet.set_column('F:F', 10)  # Score
worksheet.set_column('G:G', 10)  # Correct?

# Write Headers
# Added 'Category' at index 4
headers = ['Image', 'Question', 'True Answer', 'Model Answer', 'Category', 'Score', 'Correct']
for col, header in enumerate(headers):
    worksheet.write(0, col, header, header_format)

# Write Data and Insert Images
row_height = 100  # Height in pixels
worksheet.set_default_row(row_height) 

for i, res in enumerate(results):
    row = i + 1  # Start from row 1
    
    # 1. Insert Image (Column A)
    image_path = res['image_path']
    if os.path.exists(image_path):
        try:
            worksheet.insert_image(row, 0, image_path, {
                'x_scale': 0.2, 
                'y_scale': 0.2, 
                'object_position': 1
            })
        except Exception:
            worksheet.write(row, 0, "Error", cell_format)
    else:
        worksheet.write(row, 0, "Not Found", cell_format)

    # 2. Write Text Data
    worksheet.write(row, 1, res['question'], cell_format)
    worksheet.write(row, 2, res['true_answer'], cell_format)
    worksheet.write(row, 3, res['model_answer'], cell_format)
    
    # Column E: Category (BLANK)
    worksheet.write(row, 4, "", cell_format)
    
    # Column F: Score
    worksheet.write(row, 5, res['similarity_score'], cell_format)
    
    # Column G: Status
    status = "Correct" if res['correct'] else "Wrong"
    fmt = status_correct if res['correct'] else status_wrong
    worksheet.write(row, 6, status, fmt)

workbook.close()
print(f"✓ Success! Excel file saved as {excel_filename}")

Creating Excel file with embedded images: bangla_vqa_results_with_images.xlsx...
✓ Success! Excel file saved as bangla_vqa_results_with_images.xlsx


In [8]:
# 3. Save HTML
print("Generating HTML report...")
html_output = """
<html>
<head>
    <title>Bangla VQA Test Results</title>
    <style>
        body { font-family: sans-serif; margin: 20px; }
        .container { display: flex; flex-wrap: wrap; gap: 20px; }
        .card { border: 1px solid #ccc; border-radius: 8px; padding: 15px; width: 400px; 
                box-shadow: 2px 2px 8px rgba(0,0,0,0.1); }
        .card img { max-width: 100%; border-radius: 4px; }
        .card p { margin: 10px 0; }
        .correct { border-left: 5px solid green; }
        .incorrect { border-left: 5px solid red; }
    </style>
</head>
<body>
    <h1>Bangla VQA Test Report</h1>
    <h2>Summary</h2>
    <p><strong>Total Samples:</strong> {N_SAMPLES}</p>
    <p><strong>Accuracy (Score > 85%):</strong> {accuracy:.2f}%</p>
    <p><strong>Average Similarity:</strong> {avg_similarity:.2f}%</p>
    <hr>
    <div class='container'>
"""

def get_image_base64(image_path):
    try:
        img = Image.open(image_path)
        buffer = BytesIO()
        img.save(buffer, format="JPEG")
        img_b64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        return f"data:image/jpeg;base64,{img_b64}"
    except Exception:
        return ""

# Note: results variable is already populated from the loop above
for res in results:
    img_src = get_image_base64(res['image_path'])
    card_class = 'correct' if res['correct'] else 'incorrect'
    
    html_output += f"""
    <div class='card {card_class}'>
        <img src='{img_src}' alt='Test Image'>
        <p><b>❓ প্রশ্ন:</b> {res['question']}</p>
        <p><b>✅ সঠিক উত্তর:</b> {res['true_answer']}</p>
        <p><b>🤖 মডেলের উত্তর:</b> {res['model_answer']}</p>
        <p><b>📊 স্কোর: {res['similarity_score']}%</b></p>
    </div>
    """

html_output = html_output.replace("{N_SAMPLES}", str(N_SAMPLES))
html_output = html_output.replace("{accuracy:.2f}", f"{accuracy:.2f}")
html_output = html_output.replace("{avg_similarity:.2f}", f"{avg_similarity:.2f}")
html_output += "</div></body></html>"

html_report_file = "bangla_vqa_report.html"
with open(html_report_file, 'w', encoding='utf-8') as f:
    f.write(html_output)

print(f"✓ Success! Interactive HTML report saved to {html_report_file}")

Generating HTML report...
✓ Success! Interactive HTML report saved to bangla_vqa_report.html
